In [ ]:
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
from holoviews import opts
import holoviews as hv
import panel as pn
from thredds_crawler.crawl import Crawl
import numpy as np
import datetime as dt
from get_data import get_urls, get_model_data, get_obs_data

In [ ]:
hv.extension('bokeh')
pn.extension()

# Use-case: Time Series

## Introduction

This file contains the code for generating a time series plot for one model for a specific site and time and variable as well as corresponding observation. (OBSERVATION TO BE ADDED WHEN THEY EXIST IN PORTAL)

The code for the functions for fetching data can be found in read_data.py

This plotting tool as been generated for the Year Of Polar Prediction site Model Intercomparison Project, YOPPsiteMIP

## Code

The following cell contains the functions responsible for plotting the data. 

This use-case has two scenarios, which are user-selectable through widgets.

- The first where all the model files for a time period are plotted in full, which leads to overlap between         their shared dates.

- The second is where only one day is selected per model (wheater it be the first,second or third day is user-selectable) and are concatinated into one non-overlapping time series. 

In [ ]:
def plot_ts_stacked(data,variable):
    plots = []
    
    for item in data.columns.get_level_values(1):
        p = hv.Curve((data.index,data[variable][item])) 
        plots.append(p)
        
    plot = hv.Overlay(plots)

    return plot

def plot_ts_concat(data,variable):
    plot = hv.Curve((data.index,data[variable]))
    return plot

The code in the following cell is responsible for the widgets that enable user-selectability. 

TO DO: populate variables based on available variables in the requested files
TO DO: add SOP choice
TO DO: populate date slider based on SOP1 or SOP2 choice

give user choice based on site, model or both?
makes sense to choose site then give model selection based on models availabe for that site.

In [ ]:
#widgets for user selection of model, plot type, site, variable, start time and date range
#most selections will be expanded in the near fututre to include all relevant variables,
#models and sites. 
#there will also be a selection of SOP1 and SOP2, currently the dates are only SOP1
types = pn.widgets.Select(
    name="types", options=['stacked','concatenated'], margin=(0, 20, 0, 0)
)
variable = pn.widgets.Select(
    name="Variables", options=['tas','snd'], margin=(0, 20, 0, 0)
)
site_name = pn.widgets.Select(
    name="Site", options=['sodankyla','tiksi'], margin=(0, 20, 0, 0)
)
model_name = pn.widgets.Select(
    name="model", options=['ifs-ecmwf','AROME-Arctic'], margin=(0, 20, 0, 0)
)

start_time = pn.widgets.Select(
    name="Start time", options=['00','12'], margin=(0, 20, 0, 0)
)

dates = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2018, 2, 2), end=dt.datetime(2018, 3, 31),
    value=(dt.datetime(2018, 2, 2), dt.datetime(2018, 3, 31))
)

concat_day = pn.widgets.Select(
    name="Model day selected", options={1:0,2:1,3:2}, margin=(0,20,0,0)
)

The following cell contains the function responsible for fetching the data and calling the plotting routines. 

In [ ]:
@pn.depends(types,variable,site_name,model_name,start_time,dates,concat_day)
def main_timeseries(types,variable,site_name,model_name,start_time,dates,concat_day):
    
    urls_mod, urls_obs = get_urls(dates[0],dates[1],start_time,model_name,site_name,variable,concat_day)
    
    data = get_model_data(types,urls_mod,concat_day)
    
    if types == 'concatenated':
        plot = plot_ts_concat(data, variable)
        
    else:
        plot = plot_ts_stacked(data, variable)  
    plot.opts(title="Timevar "+variable+" "+model_name+" "+site_name,
          width=1200,
          height=400,
          ylabel=variable,
          xlabel="time: "+data.index[0].strftime("%Y-%-m-%d")+ " - " +data.index[-1].strftime("%Y-%-m-%d")
         )
    return plot

The follwing cell contains the function responsible for deploying the application

In [ ]:
pn.Column("timevar 1 sop 1", main_timeseries,pn.Row(pn.Column(site_name,model_name,variable),pn.Column(types,start_time),pn.Column(dates,concat_day)),width_policy="max").servable()